In [1]:
import asyncio
import time
import os

import asyncio_actor


In [2]:
class MyActor(asyncio_actor.Actor):
    def __init__(self, name: str) -> None:
        super().__init__()
        self.__name: str = name

    @asyncio_actor.actor_method()
    async def get_name(self) -> str:
        return self.__name

    @asyncio_actor.actor_method()
    async def set_name(self, name: str) -> None:
        self.__name = name

    @asyncio_actor.actor_method()
    async def heavy_calc(self, depth: int) -> int:
        result: int = 0
        for i in range(depth):
            result += i
        return result


MyActorSpawner = asyncio_actor.ActorSpawner(MyActor)


display(MyActor.get_name)
display(MyActor.get_name.__dict__)


print("##################")
async with MyActor("1 the name") as actor:
    print(ret := await actor.get_name())
    assert ret == "1 the name", f"actual name: '{ret}'"

    print(ret := await actor.set_name("1 bob"))
    assert ret is None, f"actual ret value: '{ret}'"

    print(ret := await actor.get_name())
    assert ret == "1 bob", f"actual name: '{ret}'"

print("##################")
async with MyActorSpawner("2 the name") as actor:
    print(ret := await actor.task(MyActor.get_name))
    assert ret == "2 the name", f"actual name: '{ret}'"

    print(ret := await actor.task(MyActor.set_name, "2 bob"))
    assert ret is None, f"actual ret value: '{ret}'"

    print(ret := await actor.task(MyActor.get_name))
    assert ret == "2 bob", f"actual name: '{ret}'"

print("##################")
async with MyActorSpawner.using_backend(asyncio_actor.backends.SubprocessBackend)("3 the name") as actor:
    print(ret := await actor.task(MyActor.get_name))
    assert ret == "3 the name", f"actual name: '{ret}'"

    print(ret := await actor.task(MyActor.set_name, "3 bob"))
    assert ret is None, f"actual ret value: '{ret}'"

    print(ret := await actor.task(MyActor.get_name))
    assert ret == "3 bob", f"actual name: '{ret}'"


{'__module__': '__main__',
 '__name__': 'get_name',
 '__qualname__': 'MyActor.get_name',
 '_ActorMethod__action': <function __main__.MyActor.get_name_wrapped(self) -> str>}

##################
1 the name
None
1 bob
##################
2 the name
None
2 bob
##################
3 the name
None
3 bob


In [4]:
async def _subtask(backend: asyncio_actor.ActorBackendFactory, idx: int):
    async with MyActorSpawner.using_backend(backend)(f"bob the {idx}") as actor:
        for round in range(10):
            result = await actor.task(MyActor.heavy_calc, depth=(round * 1_000_000))
            print(f"{idx:02d}:{round} = {result}")
            await asyncio.sleep(0)


begin = time.monotonic()
async with asyncio.TaskGroup() as _tg:
    for _i in range(5):
        _tg.create_task(_subtask(asyncio_actor.backends.DummyBackend, _i))
print(f"elapsed: {time.monotonic() - begin}s")  # elapsed: 8.14710775599815s

begin = time.monotonic()
async with asyncio.TaskGroup() as _tg:
    for _i in range(5):
        _tg.create_task(_subtask(asyncio_actor.backends.SubprocessBackend, _i))
print(f"elapsed: {time.monotonic() - begin}s")  # elapsed: 2.4023167839986854s


00:0 = 0
01:0 = 0
02:0 = 0
03:0 = 0
04:0 = 0
00:1 = 499999500000
01:1 = 499999500000
02:1 = 499999500000
03:1 = 499999500000
04:1 = 499999500000
00:2 = 1999999000000
01:2 = 1999999000000
02:2 = 1999999000000
03:2 = 1999999000000
04:2 = 1999999000000
00:3 = 4499998500000
01:3 = 4499998500000
02:3 = 4499998500000
03:3 = 4499998500000
04:3 = 4499998500000
00:4 = 7999998000000
01:4 = 7999998000000
02:4 = 7999998000000
03:4 = 7999998000000
04:4 = 7999998000000
00:5 = 12499997500000
01:5 = 12499997500000
02:5 = 12499997500000
03:5 = 12499997500000
04:5 = 12499997500000
00:6 = 17999997000000
01:6 = 17999997000000
02:6 = 17999997000000
03:6 = 17999997000000
04:6 = 17999997000000
00:7 = 24499996500000
01:7 = 24499996500000
02:7 = 24499996500000
03:7 = 24499996500000
04:7 = 24499996500000
00:8 = 31999996000000
01:8 = 31999996000000
02:8 = 31999996000000
03:8 = 31999996000000
04:8 = 31999996000000
00:9 = 40499995500000
01:9 = 40499995500000
02:9 = 40499995500000
03:9 = 40499995500000
04:9 = 40499

In [9]:
class MyActorL1(asyncio_actor.Actor):
    def __init__(self, name: str) -> None:
        super().__init__()
        self.__name: str = name

    @asyncio_actor.actor_method()
    async def get_name(self) -> str:
        return f"{self.__name}-{os.getpid()}"


MyActorL1Spawner = (
    asyncio_actor.ActorSpawner(MyActorL1)
    .using_backend(asyncio_actor.backends.SubprocessBackend)
)


class MyActorL2(asyncio_actor.Actor):
    def __init__(self, name: str) -> None:
        super().__init__()
        self.__name: str = name

    @asyncio_actor.actor_method()
    async def get_name(self) -> str:
        async with MyActorL1Spawner(self.__name) as actor:
            name = await actor.task(MyActorL1.get_name)
            return f"{name}-{os.getpid()}"

    @asyncio_actor.actor_method()
    async def fail(self) -> None:
        assert 0 == 1, "faild"

    @asyncio_actor.actor_method()
    async def raise_type(self, et: type[BaseException]) -> None:
        raise et()


MyActorL2Spawner = (
    asyncio_actor.ActorSpawner(MyActorL2)
    .using_backend(asyncio_actor.backends.SubprocessBackend)
)


async with MyActorL2Spawner("the cat") as _actor:
    _name = await _actor.task(MyActorL2.get_name)
    print(f"{_name}-{os.getpid()}")


the cat-54820-54808-46408


In [8]:
async with MyActorL2Spawner("the frog") as _actor:
    await _actor.task(MyActorL2.fail)

AssertionError: faild

In [12]:
async with MyActorL2Spawner("the frog") as _actor:
    try:
        await _actor.task(MyActorL2.raise_type, asyncio.CancelledError)
    except asyncio.CancelledError as e:
        print(f"captured: '{e}' @ asyncio.CancelledError branch")
        captured = True
    except BaseException as e:
        print(f"captured: '{e}' @ BaseException branch")
        captured = e
    else:
        print("no exceptions were thrown")
        captured = False

    assert captured is True, f"actual capturing status: {captured}"


captured: '' @ asyncio.CancelledError branch
